<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/intermediate-results/notebooks/4.0b-mb-gbm_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --user catboost==1.1
!pip install --user gcsfs
# !pip install ipywidgets==8.0.2
!pip install --user  numpy==1.23.4
!pip install --user pandas==1.5.1
!pip install --user scikit-learn==1.1.3
!pip install --user fastparquet
!pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension

You should consider upgrading via the '/opt/bwhpc/common/jupyter/base/2022-03-30/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached gcsfs-2022.11.0-py2.py3-none-any.whl (26 kB)
  Using cached google_cloud_storage-2.7.0-py2.py3-none-any.whl (110 kB)
  Using cached google_auth-2.15.0-py2.py3-none-any.whl (177 kB)
  Using cached google_auth_oauthlib-0.7.1-py2.py3-none-any.whl (19 kB)
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached google_resumable_media-2.4.0-py2.py3-none-any.whl (77 kB)
  Using cached protobuf-4.21.11-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
  Using cached googl

Run `pip install .` first to install all dependencies.

In [2]:
from catboost import CatBoostClassifier, Pool, metrics

#import gcsfs
# import google.auth
# from google.colab import auth, output

import numpy as np
import pandas as pd

import os
from pathlib import Path

from sklearn.metrics import accuracy_score

#from tqdm.notebook import tqdm
# import wandb

from typing import List, Optional

In [3]:
features_classical_size = [
    'TRADE_PRICE', 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',
       'rel_ask_size_ex', 'TRADE_SIZE', 'bid_size_ex', 'ask_size_ex',
       'rel_ask_ex', 'rel_bid_ex', 'BEST_rel_bid', 'BEST_rel_ask',
       'bid_ask_ratio_ex', 'chg_ex_lead', 'chg_ex_lag', 'chg_all_lead',
       'chg_all_lag', 'ask_ex', 'bid_ex', 'BEST_ASK', 'BEST_BID',
       'price_all_lag', 'price_all_lead', 'price_ex_lag', 'price_ex_lead'
]

In [4]:
# output_path = (
#     f"gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/train_set_60.parquet"
# )

# output_path = (
#     f"gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/val_set_20.parquet"
# )

X_train = pd.read_parquet("../artifacts/classical_size_features_log_normalized:v0/train_set_60.parquet", engine="pyarrow")
y_train = X_train["buy_sell"]
X_train = X_train[features_classical_size]

X_val = pd.read_parquet("../artifacts/classical_size_features_log_normalized:v0/test_set_20.parquet", engine="pyarrow")
y_val = X_val["buy_sell"]
X_val = X_val[features_classical_size]

y_train[y_train < 0] = 0
y_val[y_val < 0] = 0

## CatBoost Baseline 🐈‍⬛

In [5]:
model = CatBoostClassifier(task_type="CPU", iterations=2000, learning_rate=0.002, custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent')
model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [ ]:
acc = model.score(X_val, y_val)

print(acc)

In [ ]:
eval_metrics = model.eval_metrics(Pool(X_val,y_val), [metrics.Accuracy()], plot=True)